In [0]:
# Databricks Notebook source

# MAGIC %md
# MAGIC # 🔧 CONFIGURAÇÃO DO AMBIENTE - EXECUTAR PRIMEIRO!

# COMMAND ----------

# MAGIC %pip install --upgrade --force-reinstall yfinance pandas numpy plotly

# COMMAND ----------

# MAGIC %md
# MAGIC ## 🚀 REINICIAR PARA APLICAR INSTALAÇÕES

# COMMAND ----------

dbutils.library.restartPython()

# COMMAND ----------

# MAGIC %md
# MAGIC ## ✅ VERIFICAR INSTALAÇÃO

# COMMAND ----------

print("🔍 Verificando instalações...")

try:
    import yfinance as yf
    print("✅ yfinance instalado!")
except ImportError as e:
    print(f"❌ yfinance não instalado: {e}")

try:
    import pandas as pd
    print("✅ pandas instalado!")
except ImportError as e:
    print(f"❌ pandas não instalado: {e}")

try:
    import numpy as np
    print("✅ numpy instalado!")
except ImportError as e:
    print(f"❌ numpy não instalado: {e}")

try:
    import plotly.graph_objects as go
    print("✅ plotly instalado!")
except ImportError as e:
    print(f"❌ plotly não instalado: {e}")

# COMMAND ----------

# MAGIC %md
# MAGIC # 📊 DASHBOARD FINANCEIRO

# COMMAND ----------

# IMPORTAR BIBLIOTECAS
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

print("🎉 Todas as bibliotecas prontas!")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. ⚙️ CONFIGURAÇÃO

# COMMAND ----------

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']

# Período de análise
periodo = "1mo"

print(f"🎯 Analisando {len(acoes)} ações")
print(f"📅 Período: {periodo}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. 📈 COLETAR DADOS

# COMMAND ----------

dados_mercado = {}

for acao in acoes:
    try:
        print(f"⬇️  Baixando {acao}...")
        ticker = yf.Ticker(acao)
        dados = ticker.history(period=periodo, auto_adjust=True)
        
        if not dados.empty:
            dados_mercado[acao] = dados
            print(f"   ✅ {acao}: {len(dados)} registros - U$ {dados['Close'].iloc[-1]:.2f}")
        else:
            print(f"   ❌ {acao}: Sem dados disponíveis")
    except Exception as e:
        print(f"   ❌ {acao}: Erro - {e}")

print(f"\n📊 Coleta concluída: {len(dados_mercado)}/{len(acoes)} ações")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. 📊 GRÁFICO DE PREÇOS

# COMMAND ----------

if dados_mercado:
    grafico = go.Figure()
    
    for acao, dados in dados_mercado.items():
        grafico.add_trace(go.Scatter(
            x=dados.index, 
            y=dados['Close'], 
            name=acao,
            line=dict(width=2)
        ))
    
    grafico.update_layout(
        title='📈 EVOLUÇÃO DOS PREÇOS',
        xaxis_title='Data',
        yaxis_title='Preço (USD)',
        height=500,
        template='plotly_white'
    )
    
    grafico.show()
else:
    print("❌ Nenhum dado disponível para gerar gráficos")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. 📋 RELATÓRIO DE ANÁLISE

# COMMAND ----------

if dados_mercado:
    print("📋 RELATÓRIO DE PERFORMANCE")
    print("=" * 50)
    
    for acao, dados in dados_mercado.items():
        preco_atual = dados['Close'].iloc[-1]
        preco_inicial = dados['Close'].iloc[0]
        variacao = ((preco_atual - preco_inicial) / preco_inicial) * 100
        
        # Emojis para indicar performance
        if variacao > 0:
            emoji = "🟢"
        elif variacao < 0:
            emoji = "🔴"
        else:
            emoji = "⚪"
        
        print(f"{emoji} {acao}:")
        print(f"   💰 Preço Atual: U$ {preco_atual:.2f}")
        print(f"   📈 Variação: {variacao:+.2f}%")
        print(f"   📅 Período: {dados.index[0].strftime('%d/%m/%Y')} a {dados.index[-1].strftime('%d/%m/%Y')}")
        print()
        
    print(f"🎯 Análise concluída para {len(dados_mercado)} ativos")
else:
    print("❌ Nenhum dado disponível para análise")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. 💾 SALVAR DADOS (VERSÃO SEGURA)

# COMMAND ----------

# MAGIC %sql
# MAGIC -- Criar schema se não existir (execução segura)
# MAGIC CREATE SCHEMA IF NOT EXISTS workspace_analytics;

# COMMAND ----------

# Salvar dados de forma segura
try:
    if dados_mercado:
        print("💾 Preparando dados para salvar...")
        
        # Preparar dados
        todos_dados = []
        for acao, dados in dados_mercado.items():
            df = dados.copy()
            df['symbol'] = acao
            df['data_coleta'] = datetime.now()
            todos_dados.append(df.reset_index())
        
        dados_finais = pd.concat(todos_dados, ignore_index=True)
        
        # Criar DataFrame Spark
        spark_df = spark.createDataFrame(dados_finais)
        
        # Salvar no schema workspace_analytics (que criamos acima)
        spark_df.write.mode("overwrite").saveAsTable("workspace_analytics.analise_mercado")
        print("✅ Dados salvos em: workspace_analytics.analise_mercado")
        
        # Mostrar preview dos dados salvos
        print("\n📊 Preview dos dados salvos:")
        display(spark.sql("SELECT symbol, COUNT(*) as registros FROM workspace_analytics.analise_mercado GROUP BY symbol"))
            
    else:
        print("❌ Nenhum dado para salvar")
        
except Exception as e:
    print(f"❌ Erro ao salvar dados: {e}")
    print("💡 Os dados foram processados com sucesso, mas não salvos no banco.")

# COMMAND ----------

# MAGIC %md
# MAGIC # 🎉 DASHBOARD CONCLUÍDO!

# COMMAND ----------

print("""
✅ DASHBOARD EXECUTADO COM SUCESSO!

📊 Funcionalidades implementadas:
- ✅ Coleta de dados do mercado
- ✅ Gráficos interativos
- ✅ Análise de performance  
- ✅ Armazenamento em banco de dados

🚀 Próximos passos:
- Adicionar mais indicadores técnicos
- Criar alertas automáticos
- Análise preditiva
""")